In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

In [2]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [6]:
glob(r'C:\workspace\City_Rock\변경후\*.xlsx')

['C:\\workspace\\City_Rock\\바꾼후\\임시1.xlsx',
 'C:\\workspace\\City_Rock\\바꾼후\\임시2.xlsx']

In [11]:
file = glob(r'C:\workspace\City_Rock\바꾼후\*.xlsx')[0]
base = pd.read_excel(file, engine='openpyxl')
base

,■ 일별 박스오피스,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,- 조회일: 2023-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,- 출처: 영화진흥위원회 통합전산망 (http://www.kobis.or.kr),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,▶ 검색조건 [ 조회기간: 2023-05-10~2023-05-17 영화구분: 전...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,●2023년 05월 17일(수),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,94,2023년 4월 수요단편극장,NaN,8000,0,8000,1,71000,1,1,...,1,1,한국,한국,NaN,NaN,NaN,기타,NaN,NaN
870,94,"안녕, 티라노: 영원히, 함께",2019-08-14 00:00:00,4000,0,0,0,1591206229,1,0,...,1,1,한국,한국,(주)미디어캐슬,(주)넥스트엔터테인먼트월드(NEW),전체관람가,애니메이션,시즈노 코분,"시영준,정혜원,이세은,이현,김지혜,박영진,김준현,사성웅,시영준,정형석,곽윤상,이동훈..."
871,94,건설현장의 여직원을 마사지로 유혹,NaN,6000,0,6000,1,6000,1,1,...,1,1,일본,일본,NaN,(주)도키엔터테인먼트,청소년관람불가,성인물(에로),니시다 류지,NaN
872,106,라스트 버스,2023-02-09 00:00:00,0,0,0,1,21799091,0,0,...,1,1,영국,영국,NaN,주식회사 블루필름웍스,12세이상관람가,드라마,질리스 맥키넌,"티모시 스폴,필리스 로건"


In [5]:
col_list1 = ['순위', '영화명', '개봉일', '매출액', '매출액점유율', 
            '매출액증감(전일대비)', '매출액증감율(전일대비) ','누적매출액 ', 
            '관객수 ', '관객수증감(전일대비) ', '관객수증감율(전일대비) ', 
            '누적관객수 ','스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', 
            '제작사 ', '배급사 ', '등급 ', '장르 ','감독 ', '배우 ']
col_list2 = ['순위', '영화명', '개봉일', '매출액', '매출액점유율', 
            '매출액증감(전일대비)', '매출액증감율(전일대비) ','누적매출액 ', 
            '관객수 ', '관객수증감(전일대비) ', '관객수증감율(전일대비) ', 
            '누적관객수 ','스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', 
            '제작사 ', '배급사 ', '등급 ', '장르 ','감독 ', '배우 ','기준일']

In [7]:
df1 = pd.DataFrame(columns = col_list2)

for file in glob(r'C:\변경후\*.xlsx'):
    base = pd.read_excel(file, engine='openpyxl')

    ii = base[base["Unnamed: 4"].isnull()].index[range(7,len(base[base["Unnamed: 4"].isnull()].index),3)]
    ii_lis = [7]

    for i in range(len(ii)):
        ii_lis.append(int(ii[i])+3)

    df2 = pd.DataFrame(columns = col_list2)

    for jj in range(len(ii_lis)):
        if jj != len(ii_lis)-1:
            df = base.iloc[ii_lis[jj]:ii_lis[jj+1]-4]
            df.columns = col_list1
            df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
            df2 = pd.concat([df,df2],axis=0)

        else:
            df = base.iloc[ii_lis[jj]:]
            df.columns = col_list1
            df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
            df2 = pd.concat([df,df2],axis=0)
    df1 = pd.concat([df1,df2],axis=0)

In [8]:
df1.head(5)

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
385,1,미션임파서블:고스트프로토콜,2011-12-15 00:00:00,1656108500,0.299,-180477000,-0.098,47709028000,211224,-21880,...,2568,미국,미국,파라마운트 픽쳐스,(주)씨제이이엔엠,15세이상관람가,액션,브래드 버드,"톰 크루즈,제레미 레너",2012-01-08
386,2,원더풀 라디오,2012-01-05 00:00:00,931526500,0.168,-153977500,-0.142,3007567000,123253,-21344,...,2174,한국,한국,"(주)영화사아이비젼,(주)대명문화공장",(주)쇼박스,15세이상관람가,"드라마,멜로/로맨스",권칠인,"이민정,이정진,이광수,서영,황보현,이범규,박지연,서미래,김병옥,조우진,김해숙,김창완...",2012-01-08
387,3,셜록홈즈 : 그림자 게임,2011-12-21 00:00:00,726023500,0.131,-123709000,-0.146,14136250500,93839,-16488,...,1558,미국,미국,NaN,워너브러더스 코리아(주),15세이상관람가,"액션,미스터리",가이 리치,"로버트 다우니 주니어,주드 로",2012-01-08
388,4,퍼펙트 게임,2011-12-21 00:00:00,443607500,0.08,-25678000,-0.055,8737863000,57278,-4009,...,1081,한국,한국,"(주)동아수출공사,밀리언 스토리,(주)다세포클럽",롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,드라마,박희곤,"조승우,양동근,최정원,임지훈,김동희,허형규,박용규,신준호,최지훈,조찬형,차현우,한준...",2012-01-08
389,5,마이 웨이,2011-12-21 00:00:00,383844500,0.069,-32556500,-0.078,15132649500,49867,-4379,...,1423,한국,"한국,일본,중국",(주)디렉터스,"에스케이플래닛(주),(주)씨제이이엔엠",15세이상관람가,"전쟁,액션,드라마",강제규,"장동건,오다기리 조,정홍재,판빙빙,강남현,조동혁,김효연,김홍근,신승훈,홍서백,김희원...",2012-01-08


In [10]:
df1.shape

(170150, 23)

In [38]:
# df1.to_csv('./2012-2016.csv', index=False)

In [40]:
df2 = pd.read_csv('./2017-2022.csv')
df2

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,너의 이름은.,2017-01-04 00:00:00,2809088900,0.325,-19291559.0,-0.007,9745292442,333008,-2317.0,...,4101,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-08
1,2,마스터,2016-12-21 00:00:00,1903655600,0.220,-132995600.0,-0.065,52966342965,220974,-15420.0,...,3186,한국,한국,영화사 집,(주)씨제이이엔엠,15세이상관람가,"액션,범죄",조의석,"이병헌,강동원,김우빈,엄지원,박해수,오달수,진경,김정우,장한별,이승용,김대현,홍성덕...",2017-01-08
2,3,패신저스,2017-01-04 00:00:00,1107502600,0.128,-160686571.0,-0.127,4176443418,129810,-18692.0,...,2272,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,"어드벤처,SF",모튼 틸덤,"크리스 프랫,제니퍼 로렌스,마이클 쉰,앤디 가르시아,로렌스 피쉬번",2017-01-08
3,4,씽,2016-12-21 00:00:00,553849300,0.064,-47311600.0,-0.079,12233162289,68237,-6449.0,...,1274,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),전체관람가,"애니메이션,뮤지컬",가스 제닝스,"매튜 매커너히,리즈 위더스푼,스칼렛 요한슨,태런 에저튼,세스 맥팔레인",2017-01-08
4,5,눈의 여왕 3: 눈과 불의 마법대결,2017-01-04 00:00:00,535436000,0.062,-12239400.0,-0.022,1828596400,67605,-2274.0,...,1163,러시아,러시아,NaN,(주)이수C&E,전체관람가,"애니메이션,어드벤처,가족,판타지",알렉세이 트시칠린,"박지윤,엄상현,남도형,박상훈,이재범",2017-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246659,90,실버벨,2016-12-15 00:00:00,9000,0.000,-40000.0,-0.816,21778000,1,-5.0,...,1,미국,미국,NaN,(재)CBS,전체관람가,코미디,해롤드 크론크,"브루스 박스라이트너,안토니오 파르가스",2017-01-07
246660,90,장난감,NaN,7000,0.000,-68000.0,-0.907,102000,1,-14.0,...,1,중국,중국,NaN,NaN,NaN,드라마,쑨위,NaN,2017-01-07
246661,90,비치온더비치,2016-12-08 00:00:00,5000,0.000,-141000.0,-0.966,10618700,1,-17.0,...,1,한국,한국,비치사,(주)로카,청소년관람불가,멜로/로맨스,정가영,"정가영,김최용준,이하윤",2017-01-07
246662,94,그레이트 뷰티,2014-06-12 00:00:00,0,0.000,0.0,1.000,333334873,0,0.0,...,1,이탈리아,"이탈리아,프랑스",NaN,(주)영화사 진진,청소년관람불가,드라마,파올로 소렌티노,토니 세르빌로,2017-01-07


In [43]:
df2['기준일'] = pd.to_datetime(df2['기준일'], format="%Y-%m-%d")
df2['기준일']

0        2017-01-08
1        2017-01-08
2        2017-01-08
3        2017-01-08
4        2017-01-08
            ...    
246659   2017-01-07
246660   2017-01-07
246661   2017-01-07
246662   2017-01-07
246663   2017-01-07
Name: 기준일, Length: 246664, dtype: datetime64[ns]

In [44]:
imsi = df2.groupby("기준일")["영화명"].count().to_frame().reset_index()
imsi['기준연도'] = imsi['기준일'].apply(lambda x:x.year)
imsi['기준월'] = imsi['기준일'].apply(lambda x:x.month)
imsi['기준연도'].unique()

array([2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

In [45]:
imsi

,기준일,영화명,기준연도,기준월
0,2017-01-01,76,2017,1
1,2017-01-02,81,2017,1
2,2017-01-03,89,2017,1
3,2017-01-04,97,2017,1
4,2017-01-05,94,2017,1
...,...,...,...,...
2186,2022-12-27,107,2022,12
2187,2022-12-28,113,2022,12
2188,2022-12-29,93,2022,12
2189,2022-12-30,89,2022,12


In [46]:
imsi['기준월'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [47]:
len(imsi['기준일'].unique())

2191

In [49]:
df2[df2['순위'] == '합계']

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
94,합계,NaN,NaN,8651387400,NaN,NaN,NaN,345144518791,1019320,NaN,...,17840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-08
178,합계,NaN,NaN,2860584400,NaN,NaN,NaN,338457357871,377135,NaN,...,16965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-09
274,합계,NaN,NaN,2976064500,NaN,NaN,NaN,340366287671,396445,NaN,...,17017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-10
362,합계,NaN,NaN,3184751100,NaN,NaN,NaN,313701909095,421212,NaN,...,17010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-11
449,합계,NaN,NaN,3175191300,NaN,NaN,NaN,216640670467,422788,NaN,...,17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246288,합계,NaN,NaN,2805690700,NaN,NaN,NaN,257384811132,373119,NaN,...,15833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-03
246386,합계,NaN,NaN,4167483100,NaN,NaN,NaN,260867444433,552978,NaN,...,17204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-04
246481,합계,NaN,NaN,3788844500,NaN,NaN,NaN,310962923729,501869,NaN,...,17245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-05
246568,합계,NaN,NaN,4665842883,NaN,NaN,NaN,316228322112,553797,NaN,...,17632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-06


In [51]:
df2[df2['순위'] == "1"]

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,너의 이름은.,2017-01-04 00:00:00,2809088900,0.325,-1.929156e+07,-0.007,9745292442,333008,-2317.0,...,4101,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-08
95,1,너의 이름은.,2017-01-04 00:00:00,1178187800,0.412,-1.630901e+09,-0.581,10923480242,154781,-178227.0,...,3882,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-09
179,1,너의 이름은.,2017-01-04 00:00:00,1141877500,0.384,-3.631030e+07,-0.031,12065357742,150777,-4004.0,...,3901,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-10
275,1,너의 이름은.,2017-01-04 00:00:00,1062600700,0.334,-7.927680e+07,-0.069,13127958442,140350,-10427.0,...,3782,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-11
363,1,너의 이름은.,2017-01-04 00:00:00,921887800,0.290,-1.407129e+08,-0.132,14049846242,121478,-18872.0,...,3356,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246199,1,마스터,2016-12-21 00:00:00,1036588100,0.369,-3.675340e+08,-0.262,46625144365,135859,-47564.0,...,5334,한국,한국,영화사 집,(주)씨제이이엔엠,15세이상관람가,"액션,범죄",조의석,"이병헌,강동원,김우빈,엄지원,박해수,오달수,진경,김정우,장한별,이승용,김대현,홍성덕...",2017-01-03
246289,1,너의 이름은.,2017-01-04 00:00:00,1032712400,0.248,1.032253e+09,2248.918,1674169900,138028,137970.0,...,2176,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-04
246387,1,너의 이름은.,2017-01-04 00:00:00,1003477000,0.265,-2.923540e+07,-0.028,2677646900,132463,-5565.0,...,2771,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-05
246482,1,너의 이름은.,2017-01-04 00:00:00,1430176183,0.307,4.266992e+08,0.425,4107823083,169402,36939.0,...,3320,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,"카미키 류노스케,카미시라이시 모네,나가사와 마사미,이치하라 에수코,유키 아오이,시마...",2017-01-06


# 크롤링

In [4]:
# 셀레니움으로 url 주소 크롬 열기
driver = webdriver.Chrome()
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do'
driver.get(url)
time.sleep(1)

# 초기 날짜 지정
start_day = datetime.strptime("2012-01-01", '%Y-%m-%d')

while True:

    start_day_str = start_day.strftime('%Y-%m-%d')

    # 조회기간 시작일 비우기
    driver.find_element(By.NAME,'sSearchFrom').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 마지막일 비우기
    driver.find_element(By.NAME,'sSearchTo').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 시작일 값 입력
    driver.find_element(By.NAME,'sSearchFrom').send_keys(start_day_str)
    time.sleep(0.5)
    driver.find_element(By.XPATH,'/html/body').click()
    time.sleep(0.5)

    # 조회 버튼 클릭
    driver.find_element(By.CSS_SELECTOR,'div.wrap_btn .btn_blue').click()
    time.sleep(2)

    # 엑셀 파일 다운로드
    driver.find_element(By.CSS_SELECTOR,'div.tar a:nth-child(1)').click()
    time.sleep(2)

    # 자바스크립트 확인
    Alert(driver).accept()
    time.sleep(3)
    
    # 시작일 변경
    start_day = start_day + timedelta(days=7)
    
    # 최종일 지정
    if start_day == datetime.strptime("2017-01-01", '%Y-%m-%d'):
        break

In [200]:
start_day

datetime.datetime(2008, 1, 6, 0, 0)

In [201]:
start_day + timedelta(days=7)

datetime.datetime(2008, 1, 13, 0, 0)

In [204]:
datetime.strptime("2006-01-01", '%Y-%m-%d') - datetime.strptime("2003-01-12", '%Y-%m-%d')

datetime.timedelta(days=1085)

In [205]:
1085/7

155.0